In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS

In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE, ConvBase, TADCEncoder
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy
from src.utils2 import Profiler

In [5]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 15
dataset = 'cifarfs'
root = '../../dataset/cifarfs'
order = False
inner_lr = 0.001
meta_lr = 0.001
reconst_loss = nn.MSELoss(reduction='none')
inner_adapt_steps_train = 5
meta_batch_size = 10
download = False

In [6]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'cifarfs'

args = Args(100, False, 0.1, 1, 1)

In [7]:
image_trans = transforms.Compose([transforms.ToTensor()])
train_tasks = gen_tasks(dataset, root, image_transforms=image_trans, download=download, mode='train',
                        n_ways=n_ways, k_shots=k_shots, q_shots=q_shots)
valid_tasks = gen_tasks(dataset, root, image_transforms=image_trans, download=download, mode='validation',
                        n_ways=n_ways, k_shots=k_shots, q_shots=q_shots)
test_tasks = gen_tasks(dataset, root, image_transforms=image_trans, download=download, mode='test',
                        n_ways=n_ways, k_shots=k_shots, q_shots=q_shots, num_tasks=600)

In [8]:
input = train_tasks.sample()[0]
input.shape

torch.Size([100, 3, 32, 32])

In [10]:
x = nn.Conv2d(3, 64, kernel_size=3, padding=1)(input)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 16, 16])

In [11]:
x = nn.Conv2d(64, 64, kernel_size=3, padding=1)(x)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 8, 8])

In [12]:
x = nn.Conv2d(64, 64, kernel_size=3, padding=1)(x)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 4, 4])

In [13]:
x = nn.Conv2d(64, 64, kernel_size=3, padding=1)(x)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 2, 2])

In [14]:
learner = CCVAE(in_channels=3, base_channels=64, n_ways=n_ways, dataset='cifarfs', task_adapt=True, task_adapt_fn='eaen', args=args)

AttributeError: 'Args' object has no attribute 'n_ways'